In [1]:
import requests
import pandas as pd
import json
import base64
import matplotlib.pyplot as plt
import tensorflow as tf
import requests
from pprint import PrettyPrinter

In [2]:
#Dataset
data = pd.read_csv('/kaggle/input/student-depression-dataset/student_depression_dataset.csv')
data.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,'5-6 hours',Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,'5-6 hours',Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,'Less than 5 hours',Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,'7-8 hours',Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,'5-6 hours',Moderate,M.Tech,Yes,1.0,1.0,No,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27901 entries, 0 to 27900
Data columns (total 18 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     27901 non-null  int64  
 1   Gender                                 27901 non-null  object 
 2   Age                                    27901 non-null  float64
 3   City                                   27901 non-null  object 
 4   Profession                             27901 non-null  object 
 5   Academic Pressure                      27901 non-null  float64
 6   Work Pressure                          27901 non-null  float64
 7   CGPA                                   27901 non-null  float64
 8   Study Satisfaction                     27901 non-null  float64
 9   Job Satisfaction                       27901 non-null  float64
 10  Sleep Duration                         27901 non-null  object 
 11  Di

In [4]:
pp = PrettyPrinter()
pp.pprint(requests.get("https://std-depression-pred-pipeline-production.up.railway.app/v1/models/student-depression-prediction-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1743182161'}]}


In [5]:
# 📌 Fitur yang digunakan dalam model
LABEL_KEY = "Depression"
NUMERIC_FEATURES = ["Academic Pressure", "Age", "CGPA", "Study Satisfaction", "Work/Study Hours", "Job Satisfaction", "Work Pressure"]
CATEGORICAL_FEATURES = ["City", "Degree", "Dietary Habits", "Family History of Mental Illness", 
                        "Financial Stress", "Gender", "Have you ever had suicidal thoughts ?", 
                        "Sleep Duration", "Profession"]

In [6]:
def prepare_instance(data):
    """
    Mengubah dictionary input menjadi serialized TF Example (Base64).
    """
    feature_spec = {}

    feature_spec["id"] = tf.train.Feature(int64_list=tf.train.Int64List(value=[int(data["id"])]))
    
    # Tambahkan fitur numerik
    for feature in NUMERIC_FEATURES:
        feature_spec[feature] = tf.train.Feature(float_list=tf.train.FloatList(value=[data[feature]]))
    
    # Tambahkan fitur kategorikal
    for feature in CATEGORICAL_FEATURES:
        feature_spec[feature] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[data[feature].encode()]))
    
    # Buat TF Example
    example = tf.train.Example(features=tf.train.Features(feature=feature_spec))
    serialized_example = example.SerializeToString()
    
    # Encode Base64
    encoded_example = base64.b64encode(serialized_example).decode()
    
    return {"examples": {"b64": encoded_example}}

In [7]:
# Input untuk prediksi
input_data = {
    "id": 28,
    "Academic Pressure": 4.0,
    "Age": 21,
    "CGPA": 3.5,
    "Study Satisfaction": 3.0,
    "Work/Study Hours": 6.0,
    "Job Satisfaction": 0.0,
    "Work Pressure": 0.0,
    "City": "Varanasi",
    "Profession": "Student",
    "Degree": "BA",
    "Dietary Habits": "Regular",
    "Family History of Mental Illness": "Yes",
    "Financial Stress": "Yes",
    "Gender": "Male",
    "Have you ever had suicidal thoughts ?": "Yes",
    "Sleep Duration": "6-7 hours"
}

In [8]:
# Siapkan payload untuk API
payload = {
    "signature_name": "serving_default",
    "instances": [prepare_instance(input_data)]
}

# URL Model API
MODEL_API_URL = "https://std-depression-pred-pipeline-production.up.railway.app/v1/models/student-depression-prediction-model:predict"

# Kirim request ke API
response = requests.post(MODEL_API_URL, json=payload)

# Cek hasil prediksi
if response.status_code == 200:
    prediction = response.json().get("predictions", [])
    print("Prediction:", prediction)
    if prediction:
        probability = prediction[0][0]
        result = "Depressed" if probability > 0.5 else "Not Depressed"
        print(f"Result: {result} (Probability: {probability:.4f})")
else:
    print("Error:", response.text)

Prediction: [[0.999004483]]
Result: Depressed (Probability: 0.9990)
